In [ ]:
# imports
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
from deepymod.analysis import load_tensorboard
from natsort import natsorted
import os
import pandas as pd

from matplotlib.gridspec import GridSpec


In [ ]:
# settings
sns.set(context='paper', style='white')
col_width = 5.5 # neurips style column width in inches
font_size = 10

In [ ]:
df = load_tensorboard("runs/sbl_run_0/")

In [ ]:
fig = plt.figure(constrained_layout=True, figsize=(0.9 * col_width, 0.2 * col_width))
gs = GridSpec(1, 3, figure=fig)

#Left
ax = fig.add_subplot(gs[0, 0])
ax.plot(df.index, df[prior_keys].ewm(alpha=0.1).mean())
ax.axhline(1e4, color='k', linestyle='--')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title('Prior')
ax.text(1e-0, 1e6, 'a', fontdict={'weight':'bold', 'size': font_size})


# Middle
ax = fig.add_subplot(gs[0, 1])
ax.axvline(df.BIC.idxmin(), linestyle='--', color='k')
ax.plot(df.index, (df.BIC - df.BIC.min()).ewm(alpha=0.1).mean())
ax.set_xscale('log')
#plt.ylim([0, 1000])
ax.set_title(r"$\Delta$ BIC")
#plt.xlim([0, 50000])
ax.set_yticks([])
ax.text(1e-0, 6.5e3, 'b', fontdict={'weight':'bold', 'size': font_size})


# Right
ax = fig.add_subplot(gs[0, 2])
ax.plot(df.index, masked_coeff_error.ewm(alpha=0.1).mean())
ax.axvline(df.BIC.idxmin(), linestyle='--', color='k')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_title('Coefficient error')
#ax.set_xlabel('Epoch', fontdict={'weight':'bold'})
ax.text(1e-0, 5, 'c', fontdict={'weight':'bold', 'size': font_size})

plt.savefig('BIC.pdf', bbox_inches='tight')

In [ ]:

ax.set_ylim([-1.2, 0.3])
ax.set_xticks([0, 5000, 10000])
ax.set_title("Coefficients SBL")
ax.text(-3000.0, 0.25, 'd', fontdict={'weight':'bold', 'size': 12})

In [ ]:
plt.axvline(df.BIC.idxmin(), linestyle='--', color='k')
plt.semilogx(df.index, (df.BIC - df.BIC.min()).ewm(alpha=0.1).mean())
#plt.ylim([0, 1000])
plt.ylabel(r"$\Delta$ BIC")
#plt.xlim([0, 50000])

In [ ]:
prior_keys = natsorted(key for key in df.keys() if key.find('alpha') == 0)
coeff_keys = natsorted(key for key in df.keys() if key.find('coeffs') == 0)
masked_coeff_keys = natsorted(key for key in df.keys() if key.find('masked_coeffs') == 0)

In [ ]:
true_coeffs = np.zeros((1, 12))
true_coeffs[0, 2] = 0.1
true_coeffs[0, 5] = -1

In [ ]:
coeff_error = np.mean(np.abs(df[coeff_keys] - true_coeffs), axis=1)
masked_coeff_error = np.mean(np.abs(df[masked_coeff_keys] - true_coeffs), axis=1)

In [ ]:
plt.semilogy(df.index, coeff_error.ewm(alpha=0.1).mean())
plt.semilogy(df.index, masked_coeff_error.ewm(alpha=0.1).mean())
plt.axvline(df.BIC.idxmin(), linestyle='--', color='k')

In [ ]:
plt.loglog(df.index, df[prior_keys].ewm(alpha=0.1).mean())

In [ ]:
df[coeff_keys].loc[coeff_error.idxmin()]

In [ ]:
df[coeff_keys].loc[df.BIC.idxmin()]

In [ ]:
df[masked_coeffs_keys].loc[df.BIC.idxmin()]

In [ ]:
plt.axvline(df.BIC.idxmin(), linestyle='--', color='k')
plt.semilogy(df.index, df.masked_reg.ewm(alpha=0.1).mean())
plt.ylim([1e-4, 3e-4])

In [ ]:
n_samples = 1000
log_ll = n_samples * (np.log(df.masked_reg) + np.log(df.mse))

In [ ]:
plt.plot(df.index, log_ll)

In [ ]:
plt.axvline(df.BIC.idxmin(), linestyle='--', color='k')
plt.plot(df.index, df.BIC.ewm(alpha=0.1).mean())
plt.plot(df.index, log_ll.ewm(alpha=0.1).mean())
plt.ylim([-13500, -12000])

In [ ]:
n_samples / np.log(n_samples)

In [ ]:
noise_levels = np.arange(0.0, 1.01, 0.10)
noise_levels[0] = 0.01  # setting first level to 0.01

In [ ]:
noise_levels

# Plot 2.0

In [ ]:
# imports
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
from deepymod.analysis import load_tensorboard
from natsort import natsorted
import os
import pandas as pd

from matplotlib.gridspec import GridSpec


In [ ]:
#loading noise dataframes
folder = '../sbl_experiments/runs/'
df_noise = []
for file in natsorted(filter(lambda x: x.find('noise') != -1, os.listdir(folder))):
    path = os.path.join(folder, file)
    df_noise.append(load_tensorboard(path))
    
#loading sparsity dataframes
df_nx = []
for file in natsorted(filter(lambda x: x.find('nx') != -1, os.listdir(folder))):
    path = os.path.join(folder, file)
    df_nx.append(load_tensorboard(path))

In [ ]:
coeff_keys = natsorted([key for key in df_noise[0].keys() if key.find('coeff') == 0])
masked_coeff_keys = natsorted([key for key in df_noise[0].keys() if key.find('masked_coeff') == 0])
prior_keys = natsorted([key for key in df_noise[0].keys() if key.find('alpha') == 0])


true_coeffs = np.zeros((1, len(coeff_keys)))
true_coeffs[:, 2] = 0.1
true_coeffs[:, 6] = -1

In [ ]:
min_BIC_noise = [df.BIC.idxmin() for df in df_noise]
min_BIC_noise_error = [np.mean(np.abs(df_noise[idx].loc[min_BIC_noise[idx]][masked_coeff_keys].to_numpy() - true_coeffs)) for idx in np.arange(len(min_BIC_noise))]

In [ ]:
true_min_noise = [np.mean(np.abs(df[masked_coeff_keys]-true_coeffs), axis=1).idxmin() for df in df_noise]
true_min_noise_error = [np.mean(np.abs(df[masked_coeff_keys]-true_coeffs), axis=1).min() for df in df_noise]

In [ ]:
plt.plot(min_BIC_noise)
plt.plot(true_min_noise)

In [ ]:
plt.plot(true_min_noise_error)
plt.plot(min_BIC_noise_error)

plt.yscale('log')

In [ ]:
min_BIC_nx = [df.BIC.idxmin() for df in df_nx]
min_BIC_nx_error = [np.mean(np.abs(df_nx[idx].loc[min_BIC_nx[idx]][masked_coeff_keys].to_numpy() - true_coeffs)) for idx in np.arange(len(min_BIC_nx))]

In [ ]:
true_min_nx = [np.mean(np.abs(df[masked_coeff_keys]-true_coeffs), axis=1).idxmin() for df in df_nx]
true_min_nx_error = [np.mean(np.abs(df[masked_coeff_keys]-true_coeffs), axis=1).min() for df in df_nx]

In [ ]:
plt.plot(min_BIC_nx)
plt.plot(true_min_nx)

In [ ]:
plt.plot(true_min_nx_error)
plt.plot(min_BIC_nx_error)

plt.yscale('log')

In [ ]:
idx = 2
plt.semilogy(np.mean(np.abs(df_nx[idx][masked_coeff_keys] - true_coeffs), axis=1))
plt.axvline(df_nx[idx].BIC.idxmin())

In [ ]:
plt.plot(df_nx[idx].BIC)

In [ ]:
df_nx[idx].loc[df_nx[idx].BIC.idxmin()][masked_coeff_keys]

In [ ]:
df_nx[idx].loc[true_min_nx[idx]][masked_coeff_keys]

In [ ]:
plt.semilogy(df_nx[idx][prior_keys])
plt.axvline(df_nx[idx].BIC.idxmin())

In [ ]:
idx=2
plt.semilogy(df_noise[idx][prior_keys])
plt.axvline(df_noise[idx].BIC.idxmin())
plt.axhline(1e4)

In [ ]:
plt.semilogy(np.mean(np.abs(df_noise[idx][masked_coeff_keys] - true_coeffs), axis=1))
plt.axvline(df_noise[idx].BIC.idxmin())

In [ ]:
plt.plot(df_noise[idx].BIC)

In [ ]:
np.mean(np.abs(df_nx[idx].loc[min_BIC_nx[idx]][coeff_keys].to_numpy() - true_coeffs))